# 深度学习中的正则化

在深度学习领域，大多数正则化策略都会对估计进行正则化。估计的正则化以**增加偏差的成本来换取减少方差的利益**。一个好的正则化策略正是一个有利的“交易”——能够在显著减少方差的同时不过度增加偏差。

在模型的训练过程中，会出现以下三种情况：

1. 模型不包括真实的数据生成过程(也就是模型没有近似数据生成背后的广义函数)——对应欠拟合和有偏差的情形；

2. 匹配真实数据生成过程——恰好拟合以及方差和偏差保持平衡的情形；

3. 除了包含真实数据生成过程，还包括许多其他可能的生成过程——过拟合以及方差过大的情形。

正则化的目标就是使得模型从第 3 种情况转化为第 2 种情况。

实践中，我们无法得知真实数据的生成过程，所以无法得知被估计的模型族是否包括生成过程。在深度学习算法中，大多数应用都是针对这样的情况，真实数据生成过程几乎都在模型族之外。所以我们会发现，在实际的深度学习场景中，最好的拟合模型(从最小化泛化误差的意义上)是一个适当正则化的大型模型。意思是说，**先过拟合也就是增大方差，然后再通过正则化过渡到方差减小偏差不显著增大的情形。**

下面我们介绍几种正则化策略。

## 参数范数惩罚

一种正则化方法是对目标函数 J 添加一个参数范数惩罚 $\Omega(\mathbf{\theta})$ 来限制模型(神经网络、线性回归或者逻辑回归)的拟合学习能力。将正则化后的目标函数记为  $\hat{J}$:

$$\hat{J}(\mathbf{\theta};\mathbf{X},\mathbf{y})=J(\mathbf{\theta};\mathbf{X},\mathbf{y})+\alpha\Omega(\mathbf{\theta})$$

其中 $\alpha\in[0,+\infty)$ 是权衡惩罚项和目标函数贡献的超参数，$\alpha$ 为 0 表示没有正则化，$\alpha$ 越大对应的正则化惩罚越大。

当我们训练 $\hat{J}$ 时，会降低目标 J 关于训练数据的误差同时减小在某些衡量标准下参赛 $\mathbf{\theta}$ 的规模。选择不同的参数范数意味着偏好不同类型的解。下面我们会讨论各种范数对模型的影响。

***
**只正则化神经网络中的权重**

* 在神经网络中，参数包括层与层之间仿射变换的权重和偏置，因为每个权重$w_{ij}$ 会指定相邻层的两个变量如何相互作用，而每个偏置只需要控制一个单变量即可，这意味着不对偏置进行正则化也不会产生太大的方差，所以我们通常只对**权重**做惩罚而不考虑偏置。我们使用 $\mathbf{w}$ 表示所有受范数惩罚影响的权重，用向量 $\mathbf{\theta}$ 表示包含权重和无须正则化的所有参数；

* 在神经网络中，有时我们会希望对网络的不同层分配不同的 $\alpha$ 系数来进行各自独立的惩罚，但这意味着寻找合适的超参数代价很大，为了减少搜索空间，我们对所有层使用相同的权重衰减因子。

### $L^2$ 参数正则化

我们称向目标函数添加一个正则项 $\Omega(\mathbf{\theta})=\frac{1}{2}||\mathbf{w}||^2$ 使权重更加接近原点的惩罚为 **权重衰减(weight decay)的 $L^2$ 参数范数惩罚**或者**岭回归**。

我们通过研究正则化前后相应目标函数的梯度变化来洞察权重衰减的正则化表现。简单起见，假定没有偏置参数，即$\mathbf{\theta}=\mathbf{w}$。

对应的 $L^2$ 参数正则化后的目标函数为：

$$\hat{J}(\mathbf{w};\mathbf{X},\mathbf{y})=J(\mathbf{w};\mathbf{X},\mathbf{y})+\frac{\alpha}{2}\mathbf{w}^T\mathbf{w}$$

对应的梯度为

$$\nabla_{\mathbf{w}}\hat{J} = \alpha\mathbf{w}+\nabla_{\mathbf{w}}J$$

使用单步梯度下降更新权重：

$$\mathbf{w}\leftarrow \mathbf{w}-\epsilon(\alpha\mathbf{w}+\nabla_{\mathbf{w}}J)$$

即

$$\mathbf{w}\leftarrow (1-\epsilon\alpha)\mathbf{w}-\nabla_{\mathbf{w}}J$$

可以看到，加入权重衰减后会引起学习规则的修改，即在每步执行梯度更新前会先收缩权重向量。下面我们分析在训练的整个过程正则化是如何影响权重向量的。

令 $\mathbf{w}^\ast$ 为 J 取得最小训练误差时的权重向量：

$$\mathbf{w}^\ast = \arg\min_{\mathbf{w}}J(\mathbf{w})$$

在 $\mathbf{w}^\ast$ 附近做二次泰勒近似，得到

$$J(\mathbf{w}) = J(\mathbf{w}^\ast)+\frac{1}{2}(\mathbf{w}-\mathbf{w}^\ast)^T\mathbf{H}(\mathbf{w}-\mathbf{w}^\ast)$$

其中 $\mathbf{H}$ 是 J 在 $\mathbf{w}^\ast$ 处计算的 Hessian 矩阵。

备注：

* 如果成本函数本身是二次的，则这是一个完美的近似，因为函数对高于二次的求导为0，所以完整的泰勒展开只到二次。

* 因为 $\mathbf{w}^\ast$ 被定义为最小值，所以梯度为 $\mathbf{0}$, 没有一阶项；另外，$\mathbf{H}$ 是半正定的。

$\mathbf{J}$ 在 $\mathbf{w}$ 处取得最小值时，其梯度为 $\mathbf{0}$：

$$\nabla_{\mathbf{w}}J(\mathbf{w}) = \mathbf{H}(\mathbf{w}-\mathbf{w}^\ast) = \mathbf{0}$$

为了研究权重衰减带来的影响，我们在上式中加入关于参数的 $L^2$ 范数的梯度，并且讨论最小化的 $\hat{J}$。我们用 $\hat{\mathbf{w}}$ 表示此时的最优点：

$$\alpha\hat{\mathbf{w}}+\mathbf{H}(\hat{\mathbf{w}}-\mathbf{w}^\ast)=\mathbf{0}$$

得到

$$\hat{\mathbf{w}}=(\mathbf{H}+\alpha\mathbf{I})^{-1}\mathbf{H}\mathbf{w}^\ast\tag{L2-1}$$

当 $\alpha$ 趋向于 0时，正则化的解 $\hat{\mathbf{w}}$ 趋向于 $\mathbf{w}$。当 $\alpha$ 增加时呢？

因为 $\mathbf{H}$ 是实对称的，所以可以分解为：

$$\mathbf{H} = \mathbf{Q}\mathbf{\Lambda} \mathbf{Q}^T$$

其中 $\mathbf{\Lambda}$ 是特征根组成的对角矩阵， $\mathbf{Q}$ 为一组特征向量的标准正交基,满足 $\mathbf{Q}\mathbf{Q}^T = \mathbf{I}$。

将这个分解带入 (L2-1) 式，得到

$$\begin{split}
\hat{\mathbf{w}}&=(\mathbf{Q}\mathbf{\Lambda} \mathbf{Q}^T+\alpha\mathbf{I})^{-1}\mathbf{Q}\mathbf{\Lambda} \mathbf{Q}^T\mathbf{w}^\ast\\
&=[\mathbf{Q}(\mathbf{\Lambda}+\alpha\mathbf{I})\mathbf{Q}^T]^{-1}\mathbf{Q}\mathbf{\Lambda} \mathbf{Q}^T\mathbf{w}^\ast\\
&=(\mathbf{Q}^T)^{-1}(\mathbf{\Lambda}+\alpha\mathbf{I})^{-1}\mathbf{Q}^{-1}\mathbf{Q}\mathbf{\Lambda} \mathbf{Q}^T\mathbf{w}^\ast\\
&=\mathbf{Q}(\mathbf{\Lambda}+\alpha\mathbf{I})^{-1}\mathbf{\Lambda} \mathbf{Q}^T\mathbf{w}^\ast
\end{split}$$

从结果来看，**权重缩减的效果就是 $\hat{\mathbf{w}}$ 沿着由 $\mathbf{H}$ 的特征向量所定义的坐标轴缩放 $\mathbf{w}^\ast$ 。**

 记 $\mathbf{H}$ 的第 i 个特征向量（$\mathbf{Q}$ 的第 i 列）为 $\mathbf{q}_i$。 具体来说，$\hat{\mathbf{w}}$ 的第 i 个分量会以因子 

$$\frac{\lambda_i}{\lambda_i+\alpha}$$

对与 $\mathbf{q}_i$ 对齐的 $\mathbf{w}^\ast$ 的相应分量进行缩放：

1. 当 $\lambda_i\ll \alpha$ 时，缩放因子接近于 1，正则化影响较小；

2. 当 $\lambda_i\gg \alpha$ 时，缩放因子接近于 0，正则化影响较大。

如下图所示。

<img src='figure/L2.png' width='500' height='400' align='left'>

总的来说：

1. **只有在显著减小目标函数的方向上(即未正则化之前原始目标函数 J 的 Hessian 矩阵的特征根比较大的特征向量对应的方向)参数会被保留地比较好；**


2. **无助于目标函数减小的方向(对应 Hessian 矩阵较小的特征值)上改变参数不会显著增加梯度，所以这种不重要方向对应的参数分量会在训练过程中因正则化而衰减接近于 $\mathbf{0}$。**

### $L^1$ 正则化

$L^1$参数范数惩罚为 

$$\Omega(\mathbf{\theta}) = \Vert\mathbf{w}\Vert_1=\sum_i |w_i|$$

对应的代价函数为

$$\hat{J}(\mathbf{w};\mathbf{X},\mathbf{y})=J(\mathbf{w};\mathbf{X},\mathbf{y})+\alpha\Vert\mathbf{w}\Vert_1$$

如同上一节对未正则化的成本函数做的二阶泰勒展开，有

$$J(\mathbf{w}) = J(\mathbf{w}^\ast)+\frac{1}{2}(\mathbf{w}-\mathbf{w}^\ast)^T\mathbf{H}(\mathbf{w}-\mathbf{w}^\ast)$$

则正则化之后对应成本函数的近似泰勒展开为：

$$\hat{J}(\mathbf{w})=J(\mathbf{w}^\ast)+\frac{1}{2}(\mathbf{w}-\mathbf{w}^\ast)^T\mathbf{H}(\mathbf{w}-\mathbf{w}^\ast)+\alpha\Vert\mathbf{w}\Vert_1\tag{L1-1}$$

由于 $L^1$ 正则化在任意的 Hessian 矩阵下无法得到直接清晰的代数表达式(??)，我们进一步简化假设 **Hessian 矩阵是对角的，即**

$$\mathbf{H}=diag(H_{1,1},\cdots,H_{n,n})$$

其中每个 $H_{i,i}>0$。

注：如果线性回归问题中的数据已被预处理(如可以使用 PCA)，去除输入特征之间的相关性，这一假设成立(??)。

在这个假设下，我们将(L1-1)的后面两项展开，得到：

$$\hat{J}(\mathbf{w})=J(\mathbf{w}^\ast)+\sum_{i}\left[\frac{1}{2}H_{i,i}(w_i-w_i^\ast)^2+\alpha|w_i|\right]\tag{L1-2}$$

对 (L1-2) 关于 $\mathbf{w}$ 求导等于 0,得到正则化后近似代价函数的最小化解：

$$w_i = sign(w_i^\ast)\max\left\{|w_i^\ast|-\frac{\alpha}{H_{i,i}},0\right\}$$

虽然我自己并没有得到这个解析解。待解释。

相比 $L^2$ 正则化，$L^1$ 正则化有可能通过足够大的 $\alpha$ 产生更**稀疏**的解，此处的稀疏性指的是最优值中的一些参数为 0。

## 作为约束的范数惩罚

对比上一节经过参数范数正则化的代价函数：

$$\hat{J}(\mathbf{\theta};\mathbf{X},\mathbf{y})=J(\mathbf{\theta};\mathbf{X},\mathbf{y})+\alpha\Omega(\mathbf{\theta})$$

假设我们将对限制以参数的约束形式出现，比如

$$\Omega(\mathbf{\theta})\leq k$$

我们可以构造 Lagrange 函数来最小化带约束的函数，即在原始的目标函数上添加一系列惩罚。每个惩罚是被称为KKT乘子的系数以及一个表示约束是否满足函数的乘积。

以上面的约束为例，我们可以构造广义 Lagrange 函数

$$L(\mathbf{\theta},\alpha;\mathbf{X},\mathbf{y})=J(\theta;\mathbf{X},\mathbf{y})+\alpha(\Omega(\mathbf{\theta})-k)$$

其中 $\alpha\geq 0$

这个约束问题的解由下式给出

$$\mathbf{\theta}^\ast = \arg\min_{\mathbf{\theta}}\max_{\alpha}L(\mathbf{\theta},\alpha)$$

在各种优化方法中， 所有正值的 $\alpha$ 都鼓励 $\Omega(\mathbf{\theta})$ 收缩:

* $\alpha$ 在 $\Omega(\mathbf{\theta})> k$ 时必须增加(这样对应的惩罚增大，整体目标函数为了快速减小，必须使$\Omega(\mathbf{\theta})$ 快速从大于 k 的位置向 k 靠近);

* 在 $\Omega(\mathbf{\theta})<k$ 时必须减小(依然有惩罚，只是力度变小)。总的来说，

为了洞察约束的影响，我们固定 $\alpha^\ast$,把这个问题当做只和 $\mathbf{\theta}$ 相关的函数：

$$\mathbf{\theta}^\ast = \arg\min_{\mathbf{\theta}}L(\mathbf{\theta},\alpha^\ast) = \arg\min_{\mathbf{\theta}}J(\theta;\mathbf{X},\mathbf{y}) + \alpha^\ast\Omega(\mathbf{\theta})$$

这和最小化 $\hat{J}$ 的正则化训练目标函数是一致的。**我们可以将参数范数惩罚看做对权重强加的约束。**相应的范数，可以看做被约束在相应的区域内( $L^2$ 对应的是球状区域，$L^1$ 对应的是正方形区域)。

通常我们不知道权重衰减系数 $\alpha^\ast$ 约束的区域大小，因为 $\alpha^\ast$ 的值不直接告诉我们 k 的值，原则上可以求解 k。

**虽然不知道约束区域的确切大小，但可以通过增加或者减小 $\alpha$ 来大致扩大或收缩约束区域：**

* 较大的 $\alpha$，将得到一个较小的约束区域，对应的惩罚是大的；

* 较小的 $\alpha$，将得到一个较大的约束区域，对应的惩罚是小的。

备注：想想上一节，无论是哪个范数，小于$\alpha$ 的特征根所在方向会被收缩趋近于0，是否为0取决于具体的范数。

***
**使用显式限制而不是惩罚**

**重投影方法：**我们可以修改下降算法，使其先计算 $J(\mathbf{\theta})$ 的下降步，然后将 $\mathbf{\theta}$ 投影到满足 $\Omega(\mathbf{\theta})< k$ 的最近点。

**使用重投影的显式限制而不是惩罚强加约束的优点：**

* **如果我们知道什么样的 k 是合适的，而不想花时间寻找对应于 此 k 处的 $\alpha$，这种方法会非常有用；**


* **惩罚可能会导致目标函数非凸而使算法陷入局部极小：**
    
    当训练神经网络时，通常表现为训练带有几个“死亡单元”的神经网络，这些单元对网络学到的函数没有大的影响，因为相邻上下层之间出入这些单元的权重都很小，当使用权重范数惩罚训练时，即使可以通过增加权重以显著减小 J，这些配置也可能是局部最优的。
  
    重投影实现的显示约束不鼓励权重接近原点，所以在这些情况下效果更好。通过重投影实现的显式约束只在权重变大并试图离开限制区域时产生作用；
    
  
* **重投影的显式约束对优化过程增加了稳定性：**

    因为当 $\alpha$ 比较大时，很可能进入正反馈，大的权重诱导产生大的梯度，使得权重获得较大的更新，如果更新持续增加权重的大小，$\mathbf{\theta}$ 会迅速增大以至于离原点很远而发生溢出。重投影的显式约束可以防止这种迭代反馈引起权重无限制持续增加的现象。


## 提前终止

当训练有足够表示能力甚至会过拟合的大模型时，训练误差会随着时间推移逐渐降低但验证集误差会再次上升。如下图所示。

<img src='figure/overfitting.png' width='500' height='400' align='left'>

解决这个问题，可以通过在验证集误差最低时获取其参数设置，对应也就获得了验证集误差更低的模型，并且在此基础上有希望获得更好的测试误差。下面我们介绍在深度学习中因为其有效性和操作上简单而最常用的正则化策略——提前终止策略。

**提前终止策略**：
    
    在每次验证集误差有所改善时，存储参数副本。直到验证集上的误差在事先指定的循环次数内没有进一步改善时，我们就终止算法并且返回历史上验证集误差最低的参数副本。

***
**提前终止的代价与优点**

提前终止是非常高效的超参数选择算法，我们此处指的超参数是训练步数。使用这个策略有两个代价和一个优点：

* 一个显著代价是**训练期间要定期评估验证集上的泛化误差。** 有以下两种解决机制：

    1. 将定期评估验证集误差并行在与主训练过程分离的机器上或者独立的CPU/GPU；
    2. 可以使用比训练集更小的验证集或降低评估验证集的频次来减小评估代价；
    

* 一个额外代价是**存储最佳的参数副本**：

    这个代价一般可忽略，可以将其储存在较慢较大的存储器上。由于最佳参数写入属于稀疏性行为并且都是在某一轮训练过程中读取，偶发的慢写入对总训练时间影响不大。
    
    
* 一个好处是**减少训练过程的计算成本**：

    除了由于限制训练的迭代次数而明显减少的计算成本，还带来了正则化的益处(不需要计算代价函数的惩罚项这种附加项的梯度)。

下面是对应的算法伪代码：

<img src='figure/tiqian.jpg' width='500' height='400' align='left'>

***
**额外训练**

提前终止需要验证集，这意味着某些训练数据不能被模型训练到。为了更好利用验证集数据，**我们可以在完成提前终止的首次训练之后，进行额外的训练。在第二轮的额外训练步骤中，所有的训练都被包含在内。**

有两个基本的策略可以用于第二轮的训练：

* 一个策略是再次初始化模型，使用所有数据再次训练，训练步数采用第一轮提前终止训练确定的最佳步数(见下图左)。具体措施分以下两种：

    1. 对参数进行相同次数的更新；
    2. 对数据集进行相同次数的遍历，因为训练集相比第一轮变大了，所以每一次遍历数据集将会更多次更新参数。 
    
    我们无法保证哪个措施更好，要具体问题具体实验。
    

* 另一个策略是保持第一轮训练获得的最佳参数副本，然后在此基础上使用全部的数据继续训练。我们可以监控验证集的平均损失函数，继续训练直到低于提前终止时的目标值(见下图右)。

    这个策略避免了重新训练模型的高成本，但表现不是那么好，比如验证集的目标值不一定能达到之前的目标值，所以这个策略甚至不能保证终止。 

<img src='figure/tiqian1.png' width='400' height='300' align='left'>
<img src='figure/tiqian2.png' width='400' height='300' align='right'>

***
**提前终止为何具有正则化效果**

提前终止具有正则化效果可以通过展示验证集误差学习曲线是一个 U 型曲线来支持之外，但真正机制是什么呢？

Bishop 和 Sjoberg and Ljung 认为**提前终止可以将优化过程的参数空间限制在初始参数值 $\mathbf{\theta}_0$ 的小邻域内。**

更具体地，假设用学习率 $\epsilon$ 进行 $\tau$ 个优化步骤(对应于 $\tau$ 个训练迭代)。可以将 $\epsilon\tau$ 作为模型有效容量的度量，假设梯度有界，限制 $\tau$、$\epsilon$ 能够限制从 $\mathbf{\theta}_0$ 到达的参数空间的大小，如下图所示。在这个意义上，$\epsilon\tau$ 和 参数范数惩罚中的权重衰减系数 $\alpha$ 有如下等价关系：

$$\alpha \approx \frac{1}{\epsilon\tau}$$

与 $L^2$ 的等价性如下图所示。

<img src='figure/early-stopl2.png' width='500' height='400' align='left'>

比较而言，提前终止比权重衰减更有优势，提前终止能够自动确定正则化的正确的位置，权重衰减需要进行多个不同超参数值的训练实验。

## 参数绑定与参数共享

## 稀疏表示

## Bagging 与其他集成方法

Bagging 是一种模型平均的集成方法。

神经网络能找到足够多的不同的解，意味着它们可以从模型平均中受益(即使所有模型都在同一数据集上训练)。神经网络中随机初始化的差异、小批量的随机选择、超参数的差异或不同输出的非确定性实现往往足以使得集成中的不同成员具有部分独立的误差。

## Dropout 集成

***
N 枚一模一样质地均匀的硬币，正面记为 1，反面记为 0, 且 $P(1)=\frac{1}{k}$。现在 N 枚硬币各抛掷一次(独立)，请问 N 枚硬币抛掷结果组成的向量 $U = (u_1,u_2,...,u_n)$ 对应的所有可能性的和：

$$\sum_{U\in\{0,1\}^n}U$$

是多少？ 

答案是 $\frac{1}{k}$.
***

### Dropout 方法介绍

**Dropout** 提供了一种接近于通用的正则化神经网络的方法，计算简单但功能强大。一种理解是，Dropout 被认为是集成大量深层神经网络的实用 Bagging 方法，也是一种最广泛使用的隐式集成方法。

Bagging 方法作为一种集成方法要训练集成中的每个模型，并且要在每个测试样本上评估多个模型。当模型是很大的神经网络时，训练和评估需要花费很多运行时间和内存。通常我们只能集成5到10个神经网络。

Dropout 训练的集成包括了**所有从基础网络除去 _*非输出单元*_  形成的子网络。** 对于一个拥有 m 个神经元（不包含输出层）的神经网络，通过去除不同的神经元所形成的子神经网络数量为 $2^m$。

如下图所示：

<img src='figure/dropout1.png' >

具体操作上，我们可以通过对一些单元的输出乘以 0 就可以视为删除了这个单元。简单起见，我们这里介绍 乘零的简单 Dropout 算法。另外，可以通过简单修改这个算法来与从网络中移除单元的其他操作结合使用。

我们回顾一下 Bagging 学习，一般是定义 k 个模型，从训练集有放回采样构造 k 个不同的数据集，在训练集 i 上训练模型 i，最后对结果进行投票或者平均。Dropout 集成的目标是在指数级($2^m$)数量的的神经网络上近似这个过程。

具体来说，在训练中使用 Dropout 时：

* 我们会使用基于有放回采样的小批量样本产生较小步长的学习算法，如随机梯度下降等。


* 每次加载一个样本，然后对神经网络中的所有输入和隐藏单元随机采样一个不同的二进制掩码（也就是对 0或者1）。对每个单元来讲，掩码是独立采样的，掩码为 1 (也就是包含相应神经元)的采样概率是 **训练开始前一个固定的超参数**。


* 通常在每一个小批量样本训练的神经网络中，**一个输入单元被包含的概率为 0.8， 一个隐藏单元被包含的概率为 0.5**。

之后，运行和一般的神经网络一样的前向传播、反向传播以及学习更新。下图说明了再 Dropout 下的前向传播。

<img src='figure/dropout2.png' width='400'>

**Dropout 的代价函数**

假设一个掩码向量 $\mathbf{\mu}$ 来表示被包含的神经元， $ J(\mathbf{\theta}, \mathbf{\mu})$ 是由参数 $\mathbf{\theta}$ 和 掩码 $\mathbf{\mu}$ 定义的模型代价。Dropout 的训练目标是最小化 $ E_{\mu}J(\mathbf{\theta}, \mathbf{\mu})$。这个期望包含多达指数级  ($2^m$) 项，但我们可以通过抽样 $\mathbf{\mu}$ 来获得梯度的无偏估计。

**Dropout 训练 与 Bagging 的不同**

Dropout 集成的训练 与 Bagging 训练过程不太一样。两种集成模式的不同在于是否**具有参数共享模式**。

在 Bagging 情况下，所有的模型独立进行训练，在 Dropout 情况下，**所有模型共享参数，其中每个模型继承父神经网络参数的不同子集，共享模式使得在可用内存下表示指数数量的模型变得可能。**

在 Dropout 集成训练中，因为通常父神经网络会很大，所以子神经网络的数量在指数级，所以全部训练是不可能的。我们在单个步骤中训练一小部分的子网络，参数共享使得剩余子网络也有好的参数设定。

除了参数共享模式外， Dropout 集成 与 Bagging 集成一样，每个子网络中遇到的训练集确实是有放回采样的原始训练集的子集。

### Dropout 预测

***

**通过采样对算术平均进行近似推断**

Bagging 集成通过投票得出一个未知样本所对应的值。那么 Dropout 集成如何进行预测呢？我们先假定 Dropout 的输出是一个概率分布(可以构造得出，参见“深度前馈神经网络”的输出单元部分的内容)。

Bagging 集成中，每个模型 i 产生一个概率分布 $p^{(i)}(y|\mathbf{x})$。集成的输出结果是这些分布的 **算术平均**：

$$\frac{1}{k}\sum_{i=1}^{k}p^{(i)}(y|\mathbf{x})$$

Dropout 情况下，通过掩码 $\mathbf{\mu}$ 定义每个子网络的概率分布 $p(y|\mathbf{x},\mathbf{\mu})$。所有掩码情况下的 **算术平均** 由下式给出：

$$\sum_{\mathbf{\mu}}p(\mathbf{\mu})p(y|\mathbf{x},\mathbf{\mu})$$

其中 $p(\mathbf{\mu})$ 是训练时采样 $\mathbf{\mu}$ 的概率分布(比如伯努利分布)

这个求和包含多达指数级的项，除非模型结构允许某种形式的简化 (目前我们无法得知深度神经网络是否允许某种可行的简化) ，否则是不可能计算的。我们<span class="girk">可以通过采样来近似推断，即平均许多掩码(比如 10-20 个掩码就足以获得不错的表现)的输出</span>。

***
**几何平均近似整个集成的结果**

除了上面的算术平均之后，一个更好的方法是可以近似整个集成的预测，且只需一个前向传播的代价。要做到这一点，**我们改用集成成员预测分布的几何平均而不是算术平均。**

多个概率分布的几何平均不一定是一个概率分布，为了保证结果是一个概率分布，我们要求没有子网络给某一事件的分配概率为 0，并重新标准化所得分布。

通过几何平均直接定义的非标准化概率分布由下式给出：

$$\tilde{p}_{ensemble}(y|\mathbf{x}) = \sqrt[2^d]{\prod_{\mathbf{\mu}}p(y|\mathbf{x},\mathbf{\mu})}$$

其中 d 是除去输出单元后的所有单元数量。

<span class="girk">为了简化介绍，我们使用均匀分布的 $\mathbf{\mu}$,但非均匀分布也是可以的（？？？）</span>。为了做出预测，我们必须重新标准化集成如下：

$$p_{ensemble}(y|\mathbf{x}) = \frac{\tilde{p}_{ensemble}(y|\mathbf{x})}{\sum_{y^{\prime}}\tilde{p}_{ensemble}(y^{\prime}|\mathbf{x})}$$

我们如何来计算这个概率呢？

关于计算 Dropout 输出的一个重要观点是， 可以通过**构造如下模型对测试样本进行测试得到的最终输出 $p(y|\mathbf{x})$ 来近似估计 $p_{ensemble}(y|\mathbf{x})$**：

    模型包含了所有的神经元，但我们将单元 i 的输出权重乘以单元 i 的包含概率。

这个修改的动机在于得到该单元输出的正确期望。

使用训练好的模型推断时，计算的代价与不使用 Dropout 是一样的，只需要在开始进行推断前将权重除以 2 就可以。我们称这种方法为 **权重比例推断原则(weight scaling inference rule)**。这个原则在深度非线性网络上的准确性还没有理论分析，但实践中表现得很好。

对于隐藏层，因为我们通常使用 $\frac{1}{2}$ 的包含概率，权重比例原则实施上相当于：

* **在上述我们构造的模型上将Dropout 训练结束后得到的权重除以 2 来进行结果推断**；
* 实现相同结果的另一种方法是，**在训练阶段将单元的状态乘以 2**,类似如下的关系:
$$y = a\frac{x}{2}; \quad 2y = ax$$

无论采用哪种方式，我们的目标是：_**确保在测试时一个单元的期望总输入与在训练时该单元的期望总输入是大致相同的（即使在训练阶段平均有近一半的神经元是缺失的）**_。

***
**对不具有非线性隐藏单元的模型族而言，权重推断原则是精确的。**下面通过一个例子来验证这件事。

考虑 softmax 函数回归分类，其中由向量 $\mathbf{v}$ 表示 n 个输入变量：

$$P(Y=y|\mathbf{v}) = softmax(\mathbf{W}^T\mathbf{v} + \mathbf{b})_{y}$$

对 n 个输入变量作用一个二值化向量 $\mathbf{d}$:

$$P(Y=y|\mathbf{v};\mathbf{d}) = softmax\left(\mathbf{W}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}\right)_{y}$$

集成预测器被定义为重新标准化之后所有集成成员预测的几何平均：

$$P_{ensemble}(Y=y|\mathbf{v}) = \frac{\tilde{p}_{ensemble}(Y=y|\mathbf{v})}{\sum_{y^{\prime}}\tilde{p}_{ensemble}(Y=y^{\prime}|\mathbf{v})}\tag{Dropout-712}$$

其中


$$\tilde{P}_{ensemble}(Y=y|\mathbf{v}) = \sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n}p(Y=y|\mathbf{v};\mathbf{d})}$$

我们简化 $\tilde{P}_{ensemble}$ 如下：

$$\begin{split}
\tilde{P}_{ensemble}(Y=y|\mathbf{v}) 
&= \sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n} p(Y=y|\mathbf{v};\mathbf{d})}\\
&= \sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n} softmax\left(\mathbf{W}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}\right)_{y}}\\
\\
&=\sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n}\frac{exp(\mathbf{W}_{y,:}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}_y)}{\sum_{y^{\prime}}exp(\mathbf{W}_{y^{\prime},:}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}_{y^{\prime})}}}\\
\\
&=\frac{\sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n}exp(\mathbf{W}_{y,:}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}_y)}}
{\sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n}\sum_{y^{\prime}}exp(\mathbf{W}_{y^{\prime},:}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}_{y^{\prime}})}}
\end{split}$$

因为 $\tilde{P}$ 将被标准化，可以忽略那些相对 y 来说保持不变的乘法：

$$\begin{split}
\tilde{P}_{ensemble}(Y=y|\mathbf{v}) 
&\propto \sqrt[2^n]{\prod_{\mathbf{d}\in\{0,1\}^n}exp(\mathbf{W}_{y,:}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}_y)}\\
&=exp\left(\frac{1}{2^n}\sum_{\mathbf{d}\in\{0,1\}^n}\mathbf{W}_{y,:}^T(\mathbf{d}\odot\mathbf{v}) + \mathbf{b}_y\right)\\
&=exp\left(\frac{1}{2}\mathbf{W}_{y,:}^T\mathbf{v}+ \mathbf{b}_y\right)
\end{split}$$

将其带入 (Dropout-712) 式可以得到一个权重为 $\frac{1}{2}\mathbf{W}$ 的 softmax 函数分类器。

权重比例推断原则在 **条件正态输出的回归网络** 以及那些**隐藏层不包含非线性的深度网络** 都是精确的，但对 **具有非线性的深度模型** 上仅仅是一个在实践上效果很好，理论上尚未有分析的近似。

### Dropout 的特点

Dropout 集成的优缺点如下：

* **计算方便**：Dropout 产生 n 个随机二进制数与状态相乘，每个样本每次更新只需要 $O(n)$ 的计算复杂度，在反向传播之前可能只需要 $O(n)$ 的存储空间来保存这些二进制数；


* **Dropout 不限制适用的模型或训练过程**： 几乎所有使用分布表示并且可以使用随机梯度下降训练的模型上都表现很好，包括 前馈神经网络、概率模型如受限玻尔兹曼机，以及循环神经网络；许多效果差不多的其他正则化策略对模型结构的限制更严格；


* **使用 Dropout 在特定模型上的单步代价是微小的，但在一个完整的系统上使用 Dropout 的代价可能是非常显著的**：

    因为 Dropout 是一个正则化技术，它减小了模型的有效容量，所以我们必须增大模型规模来抵消这种影响。有时候我们是以更大的模型和更多训练算法的迭代次数为代价来换取使用 Dropout 在验证集上的低误差。当数据集很大时，使用正则化对于泛化误差的降低作用是微弱的，这个时候使用 Dropout 和更大模型计算代价会超过正则化带来的好处；
    
    
* **只有极少的训练样本可用时， Dropout 不会很有效**

***
关于 Dropout 的几个认知视角：

* 通过随机行为训练网络并平均多个随机决策来进行预测，实现了一种参数共享的 Bagging 形式；


* 更进一步的另一个视角是，Dropout 不仅仅是训练一个 Bagging 的集成模型，而且是共享隐藏单元的集成模型。这意味着，无论其他隐藏单元是否在模型中，隐藏单元必须都能够表现良好，也就是说，任意若干个隐藏单元组合在一起也要表现良好。 

    **Hinton 由生物学收到启发：有性繁殖涉及两个不同有机体之间交换基因，进化的压力使得基因不仅对个体自身是表现良好的，还要准备好不同个体之间的交换。这样的基因和这些特点对环境的变化是非常稳健的。**
    
    因此，Dropout 正则化每个隐藏单元不仅是适应一种情况下的良好特征，还是适应许多情况，具有很好的泛化能力的良好特征。
    
    
* Dropout 强大的大部分原因来自施加到隐藏单元的掩码噪声。可以看做是对输入内容的信息高度智能化、自适应破坏的一种形式；


* Dropout 的另一个重要方面是噪声是乘性的。

## 对抗训练

在一些精度达到人类水平的神经网络上构造数据点，比如在输入点 $\mathbf{x^\prime}$ 与附近数据点 $\mathbf{x}$ 的输出非常不同。所以对于一些原始样本和原始样本附近的**对抗样本**，人类会做出正确的判断，但网络有时候会做出错误的预测。

Goodfellow 指出，对抗样本的主要原因之一是**过度线性**。神经网络主要是基于线性块构建的，如果一个线性函数有许多输入，它的值在每个输入改变 $\epsilon$ 时，权重为 $\mathbf{w}$ 的线性函数会改变 $\epsilon||\mathbf{w}||_1$ 之多，如果 $\mathbf{w}$ 是高维的，会是一个非常大的数。

**对抗训练通过鼓励网络在训练数据附近的局部区域恒定来限制这一高度敏感的局部线性行为。可以看做一种明确地向监督神经网络引入局部恒定先验的方法。神经网络能够将函数从接近线性转化为局部近似恒定，从而可以灵活捕获到训练数据中的线性趋势的同时抵抗局部扰动。**